<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

Installing required libraries, if they are not present.

In [7]:
## !pip install beautifulsoup4
## !pip install lxml
## !pip install requests

In [5]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

Importing stuff we need.

In [6]:
html_file = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_file, 'lxml')

We use BeauifulSoup to get the contents of the Wikipedia page.

In [7]:
table = soup.find('table', class_='wikitable sortable').tbody

csv_file = open('cms_scrape.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighbourhood'])

for items in soup.find('table', class_='wikitable sortable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        postcode = data[0].text
        borough = data[1].text
        neighbourhood = data[1].find_next_sibling().text
    except IndexError:pass
    
    
    if ( borough != 'Not assigned' ):
        x = 'Not assigned'
        if ( x in neighbourhood ):
            neighbourhood = borough
            csv_writer.writerow([postcode, borough, neighbourhood])
        else:
            csv_writer.writerow([postcode, borough, neighbourhood])

csv_file.close()

We identify the table from the HTML, then write the contents to a .CSV file.

In [9]:
uniques = data['PostalCode'].unique()
data2 = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'], index=uniques)
data2.head()

,PostalCode,Borough,Neighbourhood
M3A,NaN,NaN,NaN
M4A,NaN,NaN,NaN
M5A,NaN,NaN,NaN
M6A,NaN,NaN,NaN
M7A,NaN,NaN,NaN


We read the .CSV into a pandas dataframe and use a .replace method to remove line breaks that were present in the Wikipedia page.

In [8]:
data = pd.read_csv("cms_scrape.csv") 
data = data.replace('\n', '', regex=True)
data.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


We create another empty dataframe and make rows with all the unique PostalCodes.

In [10]:
for row in data.itertuples():
    if ( pd.isnull(data2.loc[row.PostalCode].Neighbourhood) ):
        data2.loc[row.PostalCode] = [row.PostalCode, row.Borough, row.Neighbourhood]
    else:
        data2.loc[row.PostalCode].Neighbourhood = data2.loc[row.PostalCode].Neighbourhood + ", " + row.Neighbourhood
    
data2 = data2.reset_index(drop=True)
data2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


We iterate through the data in the first dataframe to populate the second one. The If-statement determines whether to use the whole row or just add another neighborhood to an existing one.

In [11]:
data2.shape

(103, 3)

And print the shape of the dataframe